# Proyecto Sprint 9


## Descripcion del proyecto y Objetivos.

## Importación de librerias, carga de archivo.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
df=pd.read_csv("/datasets/Churn.csv")

## Revisión del DF

In [ ]:
print(df.head(5))
print()
print(df.info())
print()
print(df.isnull().sum())
print()
print(df.duplicated().sum())
print()
print(df.describe())

In [ ]:
df.columns = df.columns.str.lower()

In [ ]:
promedio = df['tenure'].mean()

df['tenure'].fillna(promedio, inplace=True)

print(df.isnull().sum())

In [ ]:
df["gender_binary"] = (df["gender"]== "Female").astype("int")
df["france_binary"] = (df["geography"] == "France").astype("int")
df["germany_binary"] = (df["geography"] == "Germany").astype("int")

df.head()

### Conclusiones respecto a los datos.
Podemos observar que el DF cuenta con 14 columnas y 10,000 filas, de igual manera no hay filas duplicadas y solo encontramos valores nulos en la columna ternure, al ser un número significativo (909) se rellenarán con un promedio de la columna al no poder prescindir de poco mas del 9% de nuestros datos, la única modificacion fue cambiar todos los nombres de las columnas a minúsculas para facilitar su manipulación.   

De igual manera se pasan a binario las columnas de Gender, así como la columna Geography; para el primer caso se identifica el género femenino con un 1, quedando 0 para genero masculino, la columna geography se cambio a binario usando dos columnas nuevas, france_binary y germany_binary, no se genera una columna spain_binary ya que este sería definido por dos ceros en las nuevas columnas, pero también para evitar problemas de multicolinearidad.  

## Examina el equilibrio de clases y Entrenamiento del modelo sin corregir el desbalance.

### Equilibrio de clases

In [ ]:
class_weight = df['exited'].value_counts(normalize=True)

print(class_weight)

colors = ['skyblue', 'salmon']
class_weight.plot(kind='bar', rot=0,color=colors, figsize=(15,8))
plt.title('Relacion de porcentaje')
plt.xlabel('Clase')
plt.ylabel('Proporción')
plt.show()

### Preparación de datos para entrenar el modelo. 

In [ ]:
target= "exited"
drop_columns = ["rownumber", "customerid", "surname", "geography", "gender"]
features = [c for c in df.columns if c not in drop_columns + [target]]

for col in features:
    df[col] = df[col].astype("float")

features

In [ ]:
df_train, df_rest = train_test_split(df, test_size = 0.40, random_state = 8)
df_valid, df_test = train_test_split(df_rest, test_size = 0.50, random_state = 8)

df_train.shape, df_valid.shape,  df_test.shape

In [ ]:
X_train = df_train[features]
y_train = df_train[target]

X_val = df_valid[features]
y_val = df_valid[target]

X_test = df_test[features]
y_test = df_test[target]

### Random Forest Classifier. 

In [ ]:
model = RandomForestClassifier(random_state=8)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred)
print(f'F1-score: {f1:.2f}')

### Logistic Regression

In [ ]:
model = LogisticRegression(random_state=8)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred)
print(f'F1-score: {f1:.2f}')

### Decision Tree Classifier

In [ ]:
model = DecisionTreeClassifier(random_state=8)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred)
print(f'F1-score: {f1:.2f}')

### Conclusiones.

1.- Podemos observar que existe un desbalance entre los datos en una relación de 4:1 

El valor 0 en la columna 'exited' representa el 79.63% de las observaciones, lo que significa que aproximadamente el 79.63% de las filas en el DataFrame tienen el valor 0 en la columna 'exited'

El valor 1 en la columna 'exited' representa el 20.37% de las observaciones, lo que significa que aproximadamente el 20.37% de las filas en el DataFrame tienen el valor 1 en la columna 'exited'.

2.- En la preparación de los datos idenfificamos como el taget la columna "exited", mientras que para los features eliminamos las columnas "rownumber", "customerid", "surname", "geography", "gender", ya que o no nos aporta ningun elemento predictivo o ya fueron tratados para poder utilizarse. 

3.- Podemos observar que el valor F1 en Random Forest Classifier se encuentra cerca del objetivo solicitado 0.57, Logistic Regression se aleja bastante con 0.13 y Decision Tree Classifier obtiene un score de 0.53.



## Corrección de desequilibrio de clases y entrenamiento de modelo. 

### Corrección de desequilibrio con Synthetic Minority Over-sampling Technique (SMOTE)

In [ ]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Verificar el balance de clases después del sobremuestreo
print('Distribución de clases antes del SMOTE:', y_train.value_counts())
print('Distribución de clases después del SMOTE:', y_train_smote.value_counts())